<a href="https://colab.research.google.com/github/gtxygyzb/NLP/blob/main/course/en/chapter6/tokenizers_in_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fast tokenizers in the QA pipeline (PyTorch)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [1]:
!pip install datasets evaluate transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 73.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.7 MB/s eta 0:00:00


In [2]:
from transformers import pipeline

question_answerer = pipeline("question-answering")
context = """
🤗 Transformers is backed by the three most popular deep learning libraries — Jax, PyTorch, and TensorFlow — with a seamless integration
between them. It's straightforward to train your models with one before loading them for inference with the other.
"""
question = "Which deep learning libraries back 🤗 Transformers?"
question_answerer(question=question, context=context)

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


{'score': 0.9802603125572205,
 'start': 78,
 'end': 106,
 'answer': 'Jax, PyTorch, and TensorFlow'}

In [3]:
long_context = """
🤗 Transformers: State of the Art NLP

🤗 Transformers provides thousands of pretrained models to perform tasks on texts such as classification, information extraction,
question answering, summarization, translation, text generation and more in over 100 languages.
Its aim is to make cutting-edge NLP easier to use for everyone.

🤗 Transformers provides APIs to quickly download and use those pretrained models on a given text, fine-tune them on your own datasets and
then share them with the community on our model hub. At the same time, each python module defining an architecture is fully standalone and
can be modified to enable quick research experiments.

Why should I use transformers?

1. Easy-to-use state-of-the-art models:
  - High performance on NLU and NLG tasks.
  - Low barrier to entry for educators and practitioners.
  - Few user-facing abstractions with just three classes to learn.
  - A unified API for using all our pretrained models.
  - Lower compute costs, smaller carbon footprint:

2. Researchers can share trained models instead of always retraining.
  - Practitioners can reduce compute time and production costs.
  - Dozens of architectures with over 10,000 pretrained models, some in more than 100 languages.

3. Choose the right framework for every part of a model's lifetime:
  - Train state-of-the-art models in 3 lines of code.
  - Move a single model between TF2.0/PyTorch frameworks at will.
  - Seamlessly pick the right framework for training, evaluation and production.

4. Easily customize a model or an example to your needs:
  - We provide examples for each architecture to reproduce the results published by its original authors.
  - Model internals are exposed as consistently as possible.
  - Model files can be used independently of the library for quick experiments.

🤗 Transformers is backed by the three most popular deep learning libraries — Jax, PyTorch and TensorFlow — with a seamless integration
between them. It's straightforward to train your models with one before loading them for inference with the other.
"""
question_answerer(question=question, context=long_context)

{'score': 0.9714871048927307,
 'start': 1892,
 'end': 1919,
 'answer': 'Jax, PyTorch and TensorFlow'}

In [4]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

model_checkpoint = "distilbert-base-cased-distilled-squad"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

inputs = tokenizer(question, context, return_tensors="pt")
outputs = model(**inputs)

In [5]:
start_logits = outputs.start_logits
end_logits = outputs.end_logits
print(start_logits.shape, end_logits.shape)
print(len(inputs.tokens()), inputs.tokens())

torch.Size([1, 67]) torch.Size([1, 67])
67 ['[CLS]', 'Which', 'deep', 'learning', 'libraries', 'back', '[UNK]', 'Transformers', '?', '[SEP]', '[UNK]', 'Transformers', 'is', 'backed', 'by', 'the', 'three', 'most', 'popular', 'deep', 'learning', 'libraries', '—', 'Jax', ',', 'P', '##y', '##T', '##or', '##ch', ',', 'and', 'Ten', '##sor', '##F', '##low', '—', 'with', 'a', 'sea', '##m', '##less', 'integration', 'between', 'them', '.', 'It', "'", 's', 'straightforward', 'to', 'train', 'your', 'models', 'with', 'one', 'before', 'loading', 'them', 'for', 'in', '##ference', 'with', 'the', 'other', '.', '[SEP]']


In [6]:
import torch

sequence_ids = inputs.sequence_ids()
print(sequence_ids)
# Mask everything apart from the tokens of the context
mask = [i != 1 for i in sequence_ids]
# Unmask the [CLS] token
mask[0] = False
print(mask)
mask = torch.tensor(mask)[None] #unsqueeze 增加一个维度
print(mask, len(mask))

start_logits[mask] = -10000
end_logits[mask] = -10000

[None, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, None]
[False, True, True, True, True, True, True, True, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True]
tensor([[False,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         Fals

In [7]:
start_probabilities = torch.nn.functional.softmax(start_logits, dim=-1)[0]
end_probabilities = torch.nn.functional.softmax(end_logits, dim=-1)[0]
print(start_probabilities.shape, end_probabilities.shape)

torch.Size([67]) torch.Size([67])


In [8]:
scores = start_probabilities[:, None] * end_probabilities[None, :]
print(scores.shape)
print(scores)

torch.Size([67, 67])
tensor([[9.4340e-13, 0.0000e+00, 0.0000e+00,  ..., 1.1023e-12, 1.6345e-12,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        ...,
        [7.8000e-14, 0.0000e+00, 0.0000e+00,  ..., 9.1136e-14, 1.3514e-13,
         0.0000e+00],
        [7.3557e-14, 0.0000e+00, 0.0000e+00,  ..., 8.5944e-14, 1.2744e-13,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]], grad_fn=<MulBackward0>)


In [9]:
scores = torch.triu(scores) # 上三角矩阵
print(scores)

tensor([[9.4340e-13, 0.0000e+00, 0.0000e+00,  ..., 1.1023e-12, 1.6345e-12,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        ...,
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 9.1136e-14, 1.3514e-13,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 1.2744e-13,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]], grad_fn=<TriuBackward0>)


In [11]:
max_index = scores.argmax().item()
print(scores.argmax(), max_index, scores.shape[1])
start_index = max_index // scores.shape[1]
end_index = max_index % scores.shape[1] #二维位置，取行列
print(start_index, end_index)
print(scores[start_index, end_index])

tensor(1576) 1576 67
23 35
tensor(0.9803, grad_fn=<SelectBackward0>)


In [14]:
inputs_with_offsets = tokenizer(question, context, return_offsets_mapping=True)
offsets = inputs_with_offsets["offset_mapping"]
print("offsets:", offsets)
start_char, _ = offsets[start_index]
_, end_char = offsets[end_index]
print("start2end char:", start_char, end_char)
answer = context[start_char:end_char]

offsets: [(0, 0), (0, 5), (6, 10), (11, 19), (20, 29), (30, 34), (35, 36), (37, 49), (49, 50), (0, 0), (1, 2), (3, 15), (16, 18), (19, 25), (26, 28), (29, 32), (33, 38), (39, 43), (44, 51), (52, 56), (57, 65), (66, 75), (76, 77), (78, 81), (81, 82), (83, 84), (84, 85), (85, 86), (86, 88), (88, 90), (90, 91), (92, 95), (96, 99), (99, 102), (102, 103), (103, 106), (107, 108), (109, 113), (114, 115), (116, 119), (119, 120), (120, 124), (125, 136), (137, 144), (145, 149), (149, 150), (151, 153), (153, 154), (154, 155), (156, 171), (172, 174), (175, 180), (181, 185), (186, 192), (193, 197), (198, 201), (202, 208), (209, 216), (217, 221), (222, 225), (226, 228), (228, 235), (236, 240), (241, 244), (245, 250), (250, 251), (0, 0)]
start2end char: 78 106


In [15]:
result = {
    "answer": answer,
    "start": start_char,
    "end": end_char,
    "score": scores[start_index, end_index],
}
print(result)

{'answer': 'Jax, PyTorch, and TensorFlow', 'start': 78, 'end': 106, 'score': tensor(0.9803, grad_fn=<SelectBackward0>)}


Try it out! 找到前k=5大score值的答案

In [20]:
def getans(start_index, end_index):
  inputs_with_offsets = tokenizer(question, context, return_offsets_mapping=True)
  offsets = inputs_with_offsets["offset_mapping"]
  start_char, _ = offsets[start_index]
  _, end_char = offsets[end_index]
  answer = context[start_char:end_char]
  result = {
    "answer": answer,
    "start": start_char,
    "end": end_char,
    "score": scores[start_index, end_index],
  }
  print(result)

k = 5  # 前五大
values, indices = torch.topk(scores.view(-1), k)
print(values, indices)
row_indices = indices // scores.shape[1]
col_indices = indices % scores.shape[1]
for i in range(k):
    value = values[i].item()
    row = row_indices[i].item()
    col = col_indices[i].item()
    getans(row, col)
    #print(f"Top-{i+1}: Value={value} at Position=({row}, {col})")

tensor([9.8026e-01, 8.2478e-03, 6.8415e-03, 1.3677e-03, 3.8109e-04],
       grad_fn=<TopkBackward0>) tensor([1576, 1577, 1107, 1570, 1710])
{'answer': 'Jax, PyTorch, and TensorFlow', 'start': 78, 'end': 106, 'score': tensor(0.9803, grad_fn=<SelectBackward0>)}
{'answer': 'Jax, PyTorch, and TensorFlow —', 'start': 78, 'end': 108, 'score': tensor(0.0082, grad_fn=<SelectBackward0>)}
{'answer': 'three most popular deep learning libraries — Jax, PyTorch, and TensorFlow', 'start': 33, 'end': 106, 'score': tensor(0.0068, grad_fn=<SelectBackward0>)}
{'answer': 'Jax, PyTorch', 'start': 78, 'end': 90, 'score': tensor(0.0014, grad_fn=<SelectBackward0>)}
{'answer': 'PyTorch, and TensorFlow', 'start': 83, 'end': 106, 'score': tensor(0.0004, grad_fn=<SelectBackward0>)}


In [16]:
inputs = tokenizer(question, long_context)
print(len(inputs["input_ids"]))

461


In [21]:
inputs = tokenizer(question, long_context, max_length=384, truncation="only_second")
print(tokenizer.decode(inputs["input_ids"]))

[CLS] Which deep learning libraries back [UNK] Transformers? [SEP] [UNK] Transformers : State of the Art NLP [UNK] Transformers provides thousands of pretrained models to perform tasks on texts such as classification, information extraction, question answering, summarization, translation, text generation and more in over 100 languages. Its aim is to make cutting - edge NLP easier to use for everyone. [UNK] Transformers provides APIs to quickly download and use those pretrained models on a given text, fine - tune them on your own datasets and then share them with the community on our model hub. At the same time, each python module defining an architecture is fully standalone and can be modified to enable quick research experiments. Why should I use transformers? 1. Easy - to - use state - of - the - art models : - High performance on NLU and NLG tasks. - Low barrier to entry for educators and practitioners. - Few user - facing abstractions with just three classes to learn. - A unified A

In [31]:
sentence = "This sentence is not too long but we are going to split it anyway."
inputs = tokenizer(
    sentence, truncation=True, return_overflowing_tokens=True, max_length=6, stride=2
)

for ids in inputs["input_ids"]:
    print(tokenizer.decode(ids))

[CLS] This sentence is not [SEP]
[CLS] is not too long [SEP]
[CLS] too long but we [SEP]
[CLS] but we are going [SEP]
[CLS] are going to split [SEP]
[CLS] to split it anyway [SEP]
[CLS] it anyway. [SEP]


In [23]:
print(inputs.keys())

dict_keys(['input_ids', 'attention_mask', 'overflow_to_sample_mapping'])


In [24]:
print(inputs["overflow_to_sample_mapping"])

[0, 0, 0, 0, 0, 0, 0]


In [47]:
sentences = [
    "This sentence is not too long but we are going to split it anyway.",
    "This sentence is shorter but will still get split.",
]
inputs = tokenizer(
    sentences, truncation=True, return_overflowing_tokens=True, max_length=6, stride=2
)

print(inputs["overflow_to_sample_mapping"])

[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]


In [62]:
inputs = tokenizer(
    question,
    long_context,
    stride=128,
    max_length=384,
    padding="longest",
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)

In [63]:
print(inputs.keys())
_ = inputs.pop("overflow_to_sample_mapping") # 删除key
print(inputs.keys())
print(inputs["offset_mapping"])
offsets = inputs.pop("offset_mapping") # 将pop的key给offsets，并删除key
print(offsets)

inputs = inputs.convert_to_tensors("pt")
print(inputs["input_ids"].shape)

dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])
dict_keys(['input_ids', 'attention_mask', 'offset_mapping'])
[[(0, 0), (0, 5), (6, 10), (11, 19), (20, 29), (30, 34), (35, 36), (37, 49), (49, 50), (0, 0), (1, 2), (3, 15), (15, 16), (17, 22), (23, 25), (26, 29), (30, 33), (34, 36), (36, 37), (39, 40), (41, 53), (54, 62), (63, 72), (73, 75), (76, 79), (79, 82), (82, 86), (87, 93), (94, 96), (97, 104), (105, 110), (111, 113), (114, 119), (120, 124), (125, 127), (128, 142), (142, 143), (144, 155), (156, 166), (166, 167), (168, 176), (177, 186), (186, 187), (188, 191), (191, 194), (194, 201), (201, 202), (203, 214), (214, 215), (216, 220), (221, 231), (232, 235), (236, 240), (241, 243), (244, 248), (249, 252), (253, 262), (262, 263), (264, 267), (268, 271), (272, 274), (275, 277), (278, 282), (283, 290), (290, 291), (291, 295), (296, 298), (298, 299), (300, 306), (307, 309), (310, 313), (314, 317), (318, 326), (326, 327), (329, 330), (331, 343), (34

In [52]:
outputs = model(**inputs)

start_logits = outputs.start_logits
end_logits = outputs.end_logits
print(start_logits.shape, end_logits.shape)

torch.Size([2, 384]) torch.Size([2, 384])


In [ ]:
sequence_ids = inputs.sequence_ids()
print(sequence_ids)
print(inputs["attention_mask"])
# Mask everything apart from the tokens of the context
mask = [i != 1 for i in sequence_ids]
# Unmask the [CLS] token
mask[0] = False
# Mask all the [PAD] tokens
mask = torch.logical_or(torch.tensor(mask)[None], (inputs["attention_mask"] == 0))

start_logits[mask] = -10000
end_logits[mask] = -10000

In [56]:
start_probabilities = torch.nn.functional.softmax(start_logits, dim=-1)
end_probabilities = torch.nn.functional.softmax(end_logits, dim=-1)

In [57]:
candidates = []
for start_probs, end_probs in zip(start_probabilities, end_probabilities):
    scores = start_probs[:, None] * end_probs[None, :]
    idx = torch.triu(scores).argmax().item()

    start_idx = idx // scores.shape[1]
    end_idx = idx % scores.shape[1]
    score = scores[start_idx, end_idx].item()
    candidates.append((start_idx, end_idx, score))

print(candidates)

[(0, 18, 0.3386702239513397), (173, 184, 0.9714869856834412)]


In [58]:
for candidate, offset in zip(candidates, offsets):
    start_token, end_token, score = candidate
    start_char, _ = offset[start_token]
    _, end_char = offset[end_token]
    answer = long_context[start_char:end_char]
    result = {"answer": answer, "start": start_char, "end": end_char, "score": score}
    print(result)

{'answer': '\n🤗 Transformers: State of the Art NLP', 'start': 0, 'end': 37, 'score': 0.3386702239513397}
{'answer': 'Jax, PyTorch and TensorFlow', 'start': 1892, 'end': 1919, 'score': 0.9714869856834412}
